## Import data

In [2]:
import pandas

In [6]:
accidents = pandas.read_csv("../datasets/project/road safety 2016/road-safety-data-accidents-2016.csv")
casualties = pandas.read_csv("../datasets/project/road safety 2016/road-safety-data-casualties-2016.csv")
vehicles = pandas.read_csv("../datasets/project/road safety 2016/road-safety-data-vehicles-2016.csv")

In [35]:
accidents

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
0,2016930000001,337976,736915,-3.009671,56.520222,Tayside,Slight,1,1,30/08/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Dry,None,None,Rural,Yes,NaN
1,2016930000007,338005,730283,-3.007620,56.460652,Tayside,Slight,2,2,14/12/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,NaN
2,2016930000009,340898,730781,-2.960794,56.465498,Tayside,Serious,1,2,22/12/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Wet or damp,None,None,Urban,Yes,NaN
3,2016930000014,340585,731660,-2.966073,56.473354,Tayside,Slight,2,1,10/10/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Dry,None,None,Urban,Yes,NaN
4,2016930000017,337373,733800,-3.018720,56.492161,Tayside,Serious,1,1,23/07/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Dry,None,None,Urban,Yes,NaN
5,2016930000019,340510,730006,-2.966913,56.458487,Tayside,Slight,2,1,05/07/2016,...,None within 50 metres,Zebra,Daylight,Fine no high winds,Dry,None,None,Urban,Yes,NaN
6,2016930000025,343271,733140,-2.922800,56.486981,Tayside,Slight,2,3,12/01/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Wet or damp,None,None,Urban,Yes,NaN
7,2016930000028,339424,731894,-2.984971,56.475308,Tayside,Slight,1,1,06/01/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Raining no high winds,Wet or damp,None,None,Urban,Yes,NaN
8,2016930000032,336587,732942,-3.031274,56.484348,Tayside,Slight,1,2,22/01/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - no lighting,Fine no high winds,Wet or damp,None,None,Rural,Yes,NaN
9,2016930000036,340528,730056,-2.966632,56.458939,Tayside,Slight,2,1,23/10/2016,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Raining no high winds,Wet or damp,None,None,Urban,Yes,NaN


## Plot data

In [31]:
import folium

In [44]:
# https://stackoverflow.com/questions/42756934/how-to-plot-lat-and-long-from-pandas-dataframe-on-folium-map-group-by-some-label
# Map centered at Dundee (or, where Google tells me Dundee is...)
colors = {'Slight': 'green', 'Serious': 'orange', 'Fatal': 'red'}
map_osm = folium.Map(location=[56.4620, -2.9707], zoom_start=11)
accidents.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], 
                                               radius=10, 
                                               fill_color=colors[row["Accident_Severity"]],
                                               fill = True,
                                               stroke=False,
                                               fill_opacity=0.5,
                                              popup = row["Accident_Index"])
                                             .add_to(map_osm), axis=1)
map_osm